In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sale_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the home sale data.
home_sale_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
home_sale_df.createOrReplaceTempView("home_sales")


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
q1= spark.sql("""
    SELECT YEAR(date) AS year_sold, FORMAT_NUMBER(ROUND(AVG(price), 2), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year_sold
    ORDER BY year_sold
""").show()


+---------+----------+
|year_sold| avg_price|
+---------+----------+
|     2019|300,263.70|
|     2020|298,353.78|
|     2021|301,819.44|
|     2022|296,363.88|
+---------+----------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
q2 = spark.sql("""
    SELECT date_built AS year_built, FORMAT_NUMBER(ROUND(AVG(price), 2), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year_built
    ORDER BY year_built
""").show()

+----------+----------+
|year_built| avg_price|
+----------+----------+
|      2010|292,859.62|
|      2011|291,117.47|
|      2012|293,683.19|
|      2013|295,962.27|
|      2014|290,852.27|
|      2015|288,770.30|
|      2016|290,555.07|
|      2017|292,676.79|
+----------+----------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

q3 = spark.sql("""
 SELECT date_built AS year_built, FORMAT_NUMBER(ROUND(AVG(price), 2), 2) AS avg_price
 FROM home_sales
 WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
 GROUP BY year_built
 ORDER BY year_built
""").show()



+----------+----------+
|year_built| avg_price|
+----------+----------+
|      2010|285,010.22|
|      2011|276,553.81|
|      2012|307,539.97|
|      2013|303,676.79|
|      2014|298,264.72|
|      2015|297,609.97|
|      2016|293,965.10|
|      2017|280,317.58|
+----------+----------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

q4 = spark.sql("""
SELECT view, FORMAT_NUMBER(ROUND(AVG(price), 2), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY CAST(view AS INT)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51|788,128.21|
|  52|733,780.26|
|  53|755,214.80|
|  54|798,684.82|
|  55|771,153.32|
|  56|718,176.40|
|  57|734,340.50|
|  58|759,764.65|
|  59|791,453.00|
|  60|754,939.65|
|  61|746,877.59|
|  62|759,150.14|
|  63|711,614.55|
|  64|767,036.67|
|  65|736,679.93|
|  66|712,475.00|
|  67|737,970.96|
|  68|716,785.44|
|  69|750,537.94|
|  70|695,865.58|
+----+----------+
only showing top 20 rows

--- 0.6848580837249756 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
res =spark.catalog.isCached('home_sales')
if res: print ('cached')
else: print ('not cached')

cached


In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

q5 = spark.sql("""
SELECT view, FORMAT_NUMBER(ROUND(AVG(price), 2), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY CAST(view AS INT)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51|788,128.21|
|  52|733,780.26|
|  53|755,214.80|
|  54|798,684.82|
|  55|771,153.32|
|  56|718,176.40|
|  57|734,340.50|
|  58|759,764.65|
|  59|791,453.00|
|  60|754,939.65|
|  61|746,877.59|
|  62|759,150.14|
|  63|711,614.55|
|  64|767,036.67|
|  65|736,679.93|
|  66|712,475.00|
|  67|737,970.96|
|  68|716,785.44|
|  69|750,537.94|
|  70|695,865.58|
+----+----------+
only showing top 20 rows

--- 0.2998685836791992 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sale_df.write.partitionBy("date_built").mode("overwrite").parquet("date_built_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
partitioned_home_sale_df = spark.read.parquet("date_built_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.
partitioned_home_sale_df.createOrReplaceTempView("partitioned_home_sale")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

q6 = spark.sql("""
SELECT view, FORMAT_NUMBER(ROUND(AVG(price), 2), 2) AS avg_price
FROM partitioned_home_sale
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY CAST(view AS INT)
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  51|788,128.21|
|  52|733,780.26|
|  53|755,214.80|
|  54|798,684.82|
|  55|771,153.32|
|  56|718,176.40|
|  57|734,340.50|
|  58|759,764.65|
|  59|791,453.00|
|  60|754,939.65|
|  61|746,877.59|
|  62|759,150.14|
|  63|711,614.55|
|  64|767,036.67|
|  65|736,679.93|
|  66|712,475.00|
|  67|737,970.96|
|  68|716,785.44|
|  69|750,537.94|
|  70|695,865.58|
+----+----------+
only showing top 20 rows

--- 0.5427412986755371 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
res =spark.catalog.isCached('home_sales')
if res: print ('cached')
else: print ('not cached')

not cached
